## Data ingestion notebook for multimodal ingest
In this notebook, we are taking a PDF consists of image and texts and ingesting into clarifai app with ready to do multimodal RAG over it.

In [18]:
import os
import sys
os.environ['CLARIFAI_PAT'] = 'YOUR_PAT'

In [ ]:
!pip install clarifai-datautils
!pip install 'unstructured[pdf] @ git+https://github.com/clarifai/unstructured.git@support_clarifai_model'
!pip install opencv-python-headless

In [ ]:
!sudo apt-get update
!sudo apt-get install -y poppler-utils tesseract-ocr libgl1-mesa-glx

In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

### Initialise the Pipeline and extractors
- By default the ImageSummarizer uses Qwen-VL Vision model for the task. If you want to use different model like **OpenAI gpt-4o**, you can use this URL - https://clarifai.com/openai/chat-completion/models/gpt-4o
- Check out the community page for more models - [Vision models](https://clarifai.com/explore/models?page=1&perPage=24&filterData=%5B%7B%22field%22%3A%22model_type_id%22%2C%22value%22%3A%5B%22multimodal-to-text%22%5D%7D%5D)

In [ ]:
from clarifai_datautils.multimodal import Pipeline
from clarifai_datautils.multimodal.pipeline.cleaners import Clean_extra_whitespace
from clarifai_datautils.multimodal.pipeline.PDF import PDFPartitionMultimodal
from clarifai_datautils.multimodal.pipeline.summarizer import ImageSummarizer

# Define the pipeline
pipeline = Pipeline(
    name='pipeline-1',
    transformations=[
        PDFPartitionMultimodal(chunking_strategy = "by_title",max_characters = 1024),
        Clean_extra_whitespace(),
        ImageSummarizer(model_url = "CLARIFAI_MODEL_URL") # Replace CLARIFAI_MODEL_URL with the model URL
    ]
)
pipeline.info()

In [ ]:
# Using SDK to upload
from clarifai.client import Dataset

dataset = Dataset(url='YOUR_DATASET_URL', pat=os.environ['CLARIFAI_PAT'])
dataset.upload_dataset(pipeline.run(files="YOUR_FILE", loader=True))